In [ ]:
#default_exp learner
from nbdev.showdoc import show_doc

In [ ]:
#export
import torch.nn as nn
import torch.nn.functional as F
import shutil
from fastai.vision.all import *
from fastcore.all import *
from scipy.stats import entropy
from sklearn.model_selection import KFold

from deepflash2.metrics import Dice_f1, Iou
from deepflash2.losses import WeightedSoftmaxCrossEntropy
from deepflash2.callbacks import ElasticDeformCallback
from deepflash2.models import get_default_shapes
from deepflash2.data import TileDataset, RandomTileDataset
from deepflash2.utils import iou, plot_results
import deepflash2.tta as tta

# Patches for the `fastai` Learner

> Imlements functions necessary to build `Learner` suitable for bioimgage segmentation

In [ ]:
#export
@patch
def apply_dropout(self:Learner):
    "If a module contains 'dropout', it will be switched to .train() mode."
    for m in self.model.modules():
        if isinstance(m, nn.Dropout):
            m.train()

In [ ]:
#export
def energy_max(e, ks=20, dim=None):
    e = torch.as_tensor(e).resize_((1,1,*e.shape))
    e = F.avg_pool2d(e, ks)
    return torch.max(e)

In [ ]:
e = np.random.randn(1024,1024)
test_close(energy_max(e, ks=100),0, eps=1e-01)

In [ ]:
#export
@patch
def predict_tiles(self:Learner, ds_idx=1, dl=None, mc_dropout=False, n_times=1, use_tta=False, 
                  tta_merge='mean', energy_T=1, energy_ks=20, padding=(0,0,0,0)): #(-52,-52,-52,-52)
    "Make predictions and reconstruct tiles, optional with dropout and/or tta applied."

    if dl is None: dl = self.dls[ds_idx].new(shuffled=False, drop_last=False)
    if use_tta: tfms=[tta.HorizontalFlip(), tta.Rotate90(angles=[90,180,270])]
    else: tfms=[]

    self.model.eval()
    if mc_dropout: self.apply_dropout()

    smx_means, energy_means, stds = [], [], []
    for data in progress_bar(dl, leave=False):
        if isinstance(data, TensorImage): images = data
        else: images, _, _ = data
        m_smx = tta.Merger()
        m_energy = tta.Merger()
        out_list_smx = []
        for t in tta.Compose(tfms):
            for _ in range(n_times):
                aug_images = t.augment_image(images)
                with torch.no_grad():
                    out = self.model(aug_images)
                out = t.deaugment_mask(out)
                if sum(padding)>0: out = F.pad(out, padding)                
                m_smx.append(F.softmax(out, dim=1))
                e = (energy_T*torch.logsumexp(out/energy_T, dim=1)) #negative energy score
                m_energy.append(e)
        
        stds.append(m_smx.result('std'))
        smx_means.append(m_smx.result()) 
        energy_means.append(m_energy.result()) 
    
    softmax_pred = torch.cat(smx_means).permute(0,2,3,1)
    smx_tiles = [x for x in softmax_pred.cpu().numpy()]

    std_pred = torch.cat(stds).permute(0,2,3,1)
    std_tiles = [x[...,0] for x in std_pred.cpu().numpy()]
    
    energy_pred = torch.cat(energy_means)#.permute(0,2,3,1)
    energy_tiles = [x for x in energy_pred.cpu().numpy()]

    smxcores = dl.reconstruct_from_tiles(smx_tiles)
    segmentations = [np.argmax(x, axis=-1) for x in smxcores]
    std_deviations = dl.reconstruct_from_tiles(std_tiles)
    energy_scores = dl.reconstruct_from_tiles(energy_tiles)
    
    if energy_ks is not None:
        energy_scores = [energy_max(e, energy_ks) for e in energy_scores]

    return smxcores, segmentations, std_deviations, energy_scores

## Ensemble Learner

In [ ]:
#export
def save_tmp(pth_tmp, files, results):
    smxs, segs, stds, enrgys = results
    pth_tmp.mkdir(exist_ok=True, parents=True)
    for i, f in progress_bar(enumerate(files), total=len(files), leave=False):
        np.savez(pth_tmp/f'{f.stem}.npz', smx=smxs[i], seg=segs[i], std=stds[i], enrgy=enrgys[i])

In [ ]:
#export
class EnsembleLearner:
    def __init__(self, files, label_fn=None, n=5, n_splits=5, pretrained=None, wd=0.001,
                 repo='matjesg/deepflash2', arch="unext50_deepflash2", random_state=42,
                 metrics=None, loss_fn=None, cbs=None, mpt=False, ds_kwargs={}, 
                 opt_func=ranger, path=None, ensemble_dir='ensemble', mw_kwargs={},
                 stats=None, item_tfms=[Brightness(max_lighting=0.1)]):
        store_attr(but='path,ensemble_dir,ds_kwargs,metrics,loss,cbs')
        self.path = Path(path) if path is not None else Path('.')
        self.ensemble_dir = self.path/ensemble_dir
        self.loss_fn = loss_fn or WeightedSoftmaxCrossEntropy(axis=1)
        self.metrics = metrics or [Dice_f1(), Iou()]
        self.cbs = cbs or [SaveModelCallback(monitor='iou'), ElasticDeformCallback, ShowGraphCallback]
        for key, value in get_default_shapes(arch).items():
            ds_kwargs.setdefault(key, value)
        self.ds_kwargs = ds_kwargs
        self.models = {}
        self._set_splits()
        self.ds = RandomTileDataset(files, label_fn=label_fn, create_weights=False, **self.mw_kwargs, **self.ds_kwargs)
        self.in_channels = self.ds.get_data(max_n=1)[0].shape[-1]
        self.stats = stats or self.ds.compute_stats()
        self.batch_tfms = Normalize.from_stats(*self.stats)
        self.df_val, self.df_ens, self.df_model = None,None,None
                 
    def _set_splits(self):
        n_splits=min(self.n_splits,len(self.files))
        kf = KFold(n_splits, shuffle=True, random_state=self.random_state)
        self.splits = {key:(self.files[idx[0]], self.files[idx[1]]) for key, idx in zip(range(1,n_splits+1), kf.split(self.files))}       
        
    def fit(self, i, epochs, lr_max=5e-3, bs=4, **kwargs):
        name = self.ensemble_dir/f'{self.arch}_model-{i}.pth'
        print(f'Starting training for {name.name}')
        files_train, files_val = self.splits[i]
        train_ds = RandomTileDataset(files_train, self.label_fn, **self.mw_kwargs, **self.ds_kwargs)
        valid_ds = TileDataset(files_val, self.label_fn, **self.mw_kwargs,**self.ds_kwargs)
        dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=bs, after_item=self.item_tfms, after_batch=self.batch_tfms)
        model = torch.hub.load(self.repo, self.arch, pretrained=self.pretrained, n_classes=dls.c, in_channels=self.in_channels, **kwargs)
        if torch.cuda.is_available(): dls.cuda(), model.cuda()
        learn = Learner(dls, model, metrics=self.metrics, wd=self.wd, loss_func=self.loss_fn, opt_func=self.opt_func, cbs=self.cbs)
        if self.mpt: learn.to_fp16()
        learn.fit_one_cycle(epochs, lr_max)
        
        print(f'Saving model at {name}')    
        name.parent.mkdir(exist_ok=True, parents=True)
        torch.save(learn.model.state_dict(), name, _use_new_zipfile_serialization=False)
        self.models[i]=name
        
    def fit_ensemble(self, epochs, skip=False, **kwargs):
        for i in range(1, self.n+1):
            if skip and (i in self.models): continue
            self.fit(i, epochs,  **kwargs)
       
    def set_n(self, n):
        for i in range(n, len(self.models)):
            self.models.pop(i+1, None)            
        self.n = n
                 
    def predict(self, files, model_no, bs=4, **kwargs):
        model_path = self.models[model_no]
        ds = TileDataset(files, **self.ds_kwargs)
        dls = DataLoaders.from_dsets(ds, batch_size=bs, after_batch=self.batch_tfms, shuffle=False, drop_last=False)
        model = torch.hub.load(self.repo, self.arch, pretrained=None, n_classes=dls.c, in_channels=self.in_channels, pre_ssl=False)
        model.load_state_dict(torch.load(model_path), strict=True)
        if torch.cuda.is_available(): dls.cuda(), model.cuda()
        learn = Learner(dls, model, loss_func=self.loss_fn)
        if self.mpt: learn.to_fp16()
        results = learn.predict_tiles(dl=dls.train, use_tta=True, **kwargs)
        pth_tmp = self.path/'.tmp'/model_path.name
        save_tmp(pth_tmp, files, results)
        return results
                               
    def get_validation_results(self, **kwargs):
        res_list = []
        for i in self.models:
            _, files_val = self.splits[i]
            res = self.predict(files_val, i, **kwargs)
            for j, f in enumerate(files_val):
                msk = self.ds.get_data(f, mask=True)[0]
                m_path = self.models[i].name
                df_tmp = pd.Series({'file' : f.name,
                        'model' :  m_path,
                        'img_path': f,
                        'msk_path': self.label_fn(f),
                        'res_path': self.path/'.tmp'/m_path/f'{f.stem}.npz',
                        'iou': iou(msk, res[1][j]),
                        'energy_max': res[3][j].numpy()})
                res_list.append(df_tmp)
        self.df_val = pd.DataFrame(res_list)
        
    def show_validation_results(self, file=None):
        if self.df_val is None: self.get_validation_results()
        df = self.df_val
        if file is not None: df = df[df.file==file]
        for _, r in df.iterrows():
            img = self.ds.get_data(r.img_path)[0]
            msk = self.ds.get_data(r.img_path, mask=True)[0]
            with open(r.res_path, 'rb') as res:
                tmp = np.load(res)
                pred = tmp['seg']
                std = tmp['std']
            plot_results(img, msk, pred, std, df=r)        
                
    def get_models(self, path=None):
        path = path or self.ensemble_dir
        models = get_files(path, extensions='.pth', recurse=False)
        models = [x for x in models if self.arch in x.name]
        if len(models)>0:
            self.models = {}
            for m in models:
                model_id = int(m.stem[-1])
                self.models[model_id] = m
            print(self.models)
        else:
            print(f'No {self.arch} models found in folder: {path}')
            
    def ensemble_results(self, files):
        pth_out = self.path/'.tmp'/f'{self.arch}_ensemble'
        pth_out.mkdir(exist_ok=True, parents=True)
        res_list = []
        for f in files:
            for m in self.models.values():
                pth_tmp = self.path/'.tmp'/m.name/f'{f.stem}.npz'
                m_smx, m_std, m_enrgy = tta.Merger(), tta.Merger(), tta.Merger()
                with open(pth_tmp, 'rb') as file:
                    tmp = np.load(file)
                    m_smx.append(tmp['smx'])
                    m_std.append(tmp['std'])
                    m_enrgy.append(tmp['enrgy'])
            smx = m_smx.result()
            seg = np.argmax(smx, axis=-1)
            enrgy = m_enrgy.result()
            np.savez(pth_out/f'{f.stem}.npz', smx=smx, seg=seg, std=m_std.result(), enrgy=enrgy)
            df_tmp = pd.Series({'file' : f.name,
                                'model' :  pth_out.name,
                                'img_path': f,
                                'res_path': pth_out/f'{f.stem}.npz',
                                'energy_max': enrgy.numpy()})
            res_list.append(df_tmp)
        return pd.DataFrame(res_list)
                            
    def get_ensemble_results(self, new_files, **kwargs):   
        res_list = []
        for i in self.models:
            res = self.predict(new_files, i, **kwargs)
            for j, f in enumerate(new_files):
                m_path = self.models[i].name
                df_tmp = pd.Series({'file' : f.name,
                                    'model_no': i, 
                                    'model' :  m_path,
                                    'img_path': f,
                                    'res_path': self.path/'.tmp'/m_path/f'{f.stem}.npz',
                                    'energy_max': res[3][j].numpy()})
                res_list.append(df_tmp)
        self.df_models = pd.DataFrame(res_list)
        self.df_ens  = self.ensemble_results(new_files)
    
    def show_ensemble_results(self, file=None, model_no=None):
        if self.df_ens is None: assert print("Please run `get_ensemble_results` first.")
        if model_no is None: df = self.df_ens
        else: df = self.df_models[df_models.model_no==model_no]
        if file is not None: df = df[df.file==file]
        for _, r in df.iterrows():
            img = self.ds.get_data(r.img_path)[0]
            with open(r.res_path, 'rb') as res:
                tmp = np.load(res)
                pred = tmp['seg']
                std = tmp['std']
            plot_results(img, pred, std, df=r)   
                
    def lr_find(self, files=None, bs=4):
        files = files or self.files
        train_ds = RandomTileDataset(files, self.label_fn, **self.mw_kwargs, **self.ds_kwargs)
        dls = DataLoaders.from_dsets(train_ds,train_ds, bs=bs)
        model = torch.hub.load(self.repo, self.arch, pretrained=self.pretrained, n_classes=dls.c, in_channels=self.in_channels)
        if torch.cuda.is_available(): dls.cuda(), model.cuda()
        learn = Learner(dls, model, metrics=self.metrics, wd=self.wd, loss_func=self.loss_fn, opt_func=self.opt_func)
        if self.mpt: learn.to_fp16()
        lr_min,lr_steep = learn.lr_find()
        print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")
        return lr_min,lr_steep     
    
    def clear_tmp(self):
        try: shutil.rmtree(self.path/'.tmp')
        except: print("No temporary files to delete")

In [ ]:
#export
add_docs(EnsembleLearner, "Meta class to train and predict model ensembles with `n` models",
         fit="Fit model number `i`",
         fit_ensemble="Fit `i` models and `skip` existing",
         predict="Predict `files` with model at `model_path`",
         get_validation_results="Validate models on validation data and save results",
         show_validation_results="Plot results of all or `file` validation images",
         ensemble_results="Merge single model results",
         get_ensemble_results="Get models and ensemble results", 
         show_ensemble_results="Show result of ensemble or `model_no`",
         get_models="Get models saved at `path`",
         set_n="Change to `n` models per ensemble",
         lr_find="Wrapper for learning rate finder",
         clear_tmp="Clear directory with temporary files"
)

In [ ]:
show_doc(EnsembleLearner)

<h2 id="EnsembleLearner" class="doc_header"><code>class</code> <code>EnsembleLearner</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>EnsembleLearner</code>(**`files`**, **`label_fn`**=*`None`*, **`n`**=*`5`*, **`n_splits`**=*`5`*, **`pretrained`**=*`None`*, **`wd`**=*`0.001`*, **`repo`**=*`'matjesg/deepflash2'`*, **`arch`**=*`'unext50_deepflash2'`*, **`random_state`**=*`42`*, **`metrics`**=*`None`*, **`loss_fn`**=*`None`*, **`cbs`**=*`None`*, **`mpt`**=*`False`*, **`ds_kwargs`**=*`{}`*, **`opt_func`**=*`ranger`*, **`path`**=*`None`*, **`ensemble_dir`**=*`'ensemble'`*, **`mw_kwargs`**=*`{}`*, **`stats`**=*`None`*, **`item_tfms`**=*`[Brightness -- {'max_lighting': 0.1, 'p': 1.0, 'draw': None, 'batch': False}:
encodes: (TensorImage,object) -> encodes
decodes: ]`*)

Meta class to train and predict model ensembles with `n` models

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_learner.ipynb.
Converted 01_models.ipynb.
Converted 02_data.ipynb.
Converted 03_metrics.ipynb.
Converted 04_callbacks.ipynb.
Converted 05_losses.ipynb.
Converted 06_utils.ipynb.
Converted 07_tta.ipynb.
Converted 08_gui.ipynb.
Converted add_information.ipynb.
Converted gt_estimation.ipynb.
Converted index.ipynb.
Converted model_library.ipynb.
Converted predict.ipynb.
Converted train-Copy1.ipynb.
Converted train.ipynb.
